In [63]:
# import the necessary libraries you need for your analysis
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import zipfile
import psycopg2

In [64]:
from dotenv import load_dotenv
load_dotenv()

True

In [65]:
#merge files into one
# List of CSV files to be combined
csv_files = ['data/April20.csv', 'data/April21.csv', 'data/April22.csv','data/Aug20.csv', 'data/Aug21.csv', 'data/Aug22.csv',
            'data/dez20.csv', 'data/Dez21.csv', 'data/DEZ22.csv', 'data/feb20.csv', 'data/Feb22.csv', 'data/Februar21.csv',
            'data/jan20.csv', 'data/Jan21.csv', 'data/Jan22.csv', 'data/Jan23.csv', 'data/jul20.csv', 'data/Jul21.csv', 
            'data/Juli22.csv', 'data/Jun21.csv', 'data/Juni20.csv', 'data/Mai20.csv', 'data/Mai21.csv', 'data/Mai22.csv', 
            'data/Mar20.csv', 'data/mar22.csv', 'data/März21.csv', 'data/Nov20.csv', 'data/Nov21.csv', 'data/Nov22.csv', 
            'data/Okt20.csv', 'data/Okt21.csv', 'data/Okt22.csv', 'data/Sep20.csv', 'data/Sep21.csv', 'data/Sept22.csv',
            'data/Sept19.csv', 'data/okt19.csv', 'data/Nov19.csv', 'data/may19.csv', 'data/Mar19.csv', 'data/jun19.csv','data/jul19.csv',
            'data/jan19.csv', 'data/feb19.csv', 'data/Dez19.csv', 'data/Aug19.csv', 'data/April2019.csv']

# Create an empty list to store the DataFrames
monthly_data = []

# Loop through the list of CSV files
for csv_file in csv_files:
    # Read the CSV file into a DataFrame
    merged_monthly = pd.read_csv(csv_file, encoding='latin-1')
    
    # Add the DataFrame to the list
    monthly_data.append(merged_monthly)

# Concatenate the list of DataFrames into a single DataFrame
months_combined = pd.concat(monthly_data)

In [66]:
months_combined.info

<bound method DataFrame.info of        index        date  federal_state_id  federal_state_name  \
0          1  15/04/2020               NaN  Schleswig-Holstein   
1          2  15/04/2020               NaN  Schleswig-Holstein   
2          3  15/04/2020               NaN  Schleswig-Holstein   
3          4  15/04/2020               NaN  Schleswig-Holstein   
4          5  15/04/2020               NaN  Schleswig-Holstein   
...      ...         ...               ...                 ...   
20821  20822  15/04/2019               NaN           Thüringen   
20822  20823  15/04/2019               NaN           Thüringen   
20823  20824  15/04/2019               NaN           Thüringen   
20824  20825  15/04/2019               NaN           Thüringen   
20825  20826  15/04/2019               NaN           Thüringen   

       professional_area_ID                        professional_area  \
0                       NaN                                   Gesamt   
1                      11.0    

In [67]:
months_combined['federal_state_id'] = np.where(months_combined['federal_state_name'] == 'Schleswig-Holstein', 1, 
                                    np.where(months_combined['federal_state_name'] == 'Hamburg', 2,
                                   np.where(months_combined['federal_state_name'] == 'Saarland', 10, 
                                   np.where(months_combined['federal_state_name'] == 'Berlin', 11,
                                   np.where(months_combined['federal_state_name'] == 'Brandenburg', 12,
                                   np.where(months_combined['federal_state_name'] == 'Mecklenburg-Vorpommern', 13,
                                  np.where(months_combined['federal_state_name'] == 'Sachsen', 14,
                                   np.where(months_combined['federal_state_name'] == 'Thüringen', 16,
                               np.where(months_combined['federal_state_name'] == 'Niedersachsen', 3,
                        np.where(months_combined['federal_state_name'] == 'Bremen', 4,
                        np.where(months_combined['federal_state_name'] == 'Nordrhein-Westfalen', 5,
                      np.where(months_combined['federal_state_name'] == 'Hessen', 6,
                    np.where(months_combined['federal_state_name'] == 'Rheinland-Pfalz', 7,
                np.where(months_combined['federal_state_name'] == 'Baden-Württemberg', 8,
                                np.where(months_combined['federal_state_name'] == 'Bayern', 9,
       np.where(months_combined['federal_state_name'] == 'Sachsen-Anhalt', 15, 0))))))))))))))))

In [68]:
months_combined.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 1022397 entries, 0 to 20825
Data columns (total 10 columns):
 #   Column                                       Non-Null Count    Dtype  
---  ------                                       --------------    -----  
 0   index                                        1022397 non-null  int64  
 1   date                                         1022397 non-null  object 
 2   federal_state_id                             1022397 non-null  int64  
 3   federal_state_name                           1022397 non-null  object 
 4   professional_area_ID                         1021629 non-null  float64
 5   professional_area                            1022397 non-null  object 
 6   number_of_unemployed                         1022397 non-null  object 
 7   number_of_job _seekers                       1022397 non-null  object 
 8   reported_open_positions                      1022389 non-null  object 
 9   reported_open_positions_moving_annual_total  102

In [69]:
months_combined.fillna(0, inplace=True)

In [43]:
#turn the columns to numeric values from object
months_combined['number_of_job _seekers'] = pd.to_numeric(months_combined['number_of_job _seekers'], errors='coerce')
months_combined['number_of_unemployed'] = pd.to_numeric(months_combined['number_of_unemployed'], errors='coerce')
months_combined['reported_open_positions'] = pd.to_numeric(months_combined['reported_open_positions'], errors='coerce')
months_combined['reported_open_positions_moving_annual_total'] = pd.to_numeric(months_combined['reported_open_positions_moving_annual_total'], errors='coerce')

In [56]:
#fill the null values with 0
months_combined.fillna(0, inplace=True)

In [45]:
#turn column values to integers
months_combined['federal_state_id'] = months_combined['federal_state_id'].astype(int)
months_combined['professional_area_ID'] = months_combined['professional_area_ID'].astype(int)
months_combined['number_of_unemployed'] = months_combined['number_of_unemployed'].astype(int)
months_combined['number_of_job _seekers'] = months_combined['number_of_job _seekers'].astype(int)
months_combined['reported_open_positions'] = months_combined['reported_open_positions'].astype(int)
months_combined['reported_open_positions_moving_annual_total'] = months_combined['reported_open_positions_moving_annual_total'].astype(int)


In [57]:
months_combined.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 1022397 entries, 0 to 20825
Data columns (total 10 columns):
 #   Column                                       Non-Null Count    Dtype  
---  ------                                       --------------    -----  
 0   index                                        1022397 non-null  int64  
 1   date                                         1022397 non-null  object 
 2   federal_state_id                             1022397 non-null  int64  
 3   federal_state_name                           1022397 non-null  object 
 4   professional_area_ID                         1022397 non-null  float64
 5   professional_area                            1022397 non-null  object 
 6   number_of_unemployed                         1022397 non-null  object 
 7   number_of_job _seekers                       1022397 non-null  object 
 8   reported_open_positions                      1022397 non-null  object 
 9   reported_open_positions_moving_annual_total  102

In [58]:
months_combined

,index,date,federal_state_id,federal_state_name,professional_area_ID,professional_area,number_of_unemployed,number_of_job _seekers,reported_open_positions,reported_open_positions_moving_annual_total
0,1,15/04/2020,1,Schleswig-Holstein,0.0,Gesamt,"40,792","69,463","18,405","50,752"
1,2,15/04/2020,1,Schleswig-Holstein,11.0,"Land-, Tier-, Forstwirtschaftsberufe",293,468,92,258
2,3,15/04/2020,1,Schleswig-Holstein,111.0,Landwirtschaft,107,175,41,116
3,4,15/04/2020,1,Schleswig-Holstein,1110.0,Berufe in der Landwirtschaft (o.S.),88,149,37,101
4,5,15/04/2020,1,Schleswig-Holstein,11102.0,Landwirtschaft (o.S.) - Fachkraft,47,79,34,82
...,...,...,...,...,...,...,...,...,...,...
20821,20822,15/04/2019,16,Thüringen,94623.0,Requisite - Spezialist,7,11,-,-
20822,20823,15/04/2019,16,Thüringen,947.0,Museumstechnik und -management,20,41,5,14
20823,20824,15/04/2019,16,Thüringen,9470.0,Museumsberufe (o.S.),16,36,5,14
20824,20825,15/04/2019,16,Thüringen,94704.0,Museum (o.S.) - Experte,16,36,5,14


In [59]:
months_combined['number_of_unemployed'].sort_values(ascending=False)

14933      999
19744      999
6201       999
9879       999
7062       999
         ...  
14831    1,000
12590    1,000
5467     1,000
15076    1,000
14586    1,000
Name: number_of_unemployed, Length: 1022397, dtype: object

In [29]:
months_combined.to_csv("data/merged_monthly.csv", sep="\t", index=False)

In [18]:
#save data as a csv file
#months_combined.to_csv('data/months_combined.csv', index= False)

In [30]:
months_combined.head()

,index,date,federal_state_id,federal_state_name,professional_area_ID,professional_area,number_of_unemployed,number_of_job _seekers,reported_open_positions,reported_open_positions_moving_annual_total
0,1,15/04/2020,1,Schleswig-Holstein,0,Gesamt,0,0,0,0
1,2,15/04/2020,1,Schleswig-Holstein,11,"Land-, Tier-, Forstwirtschaftsberufe",293,468,92,258
2,3,15/04/2020,1,Schleswig-Holstein,111,Landwirtschaft,107,175,41,116
3,4,15/04/2020,1,Schleswig-Holstein,1110,Berufe in der Landwirtschaft (o.S.),88,149,37,101
4,5,15/04/2020,1,Schleswig-Holstein,11102,Landwirtschaft (o.S.) - Fachkraft,47,79,34,82


In [31]:
months_combined.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 1022397 entries, 0 to 20825
Data columns (total 10 columns):
 #   Column                                       Non-Null Count    Dtype 
---  ------                                       --------------    ----- 
 0   index                                        1022397 non-null  int64 
 1   date                                         1022397 non-null  object
 2   federal_state_id                             1022397 non-null  int64 
 3   federal_state_name                           1022397 non-null  object
 4   professional_area_ID                         1022397 non-null  int64 
 5   professional_area                            1022397 non-null  object
 6   number_of_unemployed                         1022397 non-null  int64 
 7   number_of_job _seekers                       1022397 non-null  int64 
 8   reported_open_positions                      1022397 non-null  int64 
 9   reported_open_positions_moving_annual_total  1022397 non-nu

In [32]:
months_combined['date'].unique()

array(['15/04/2020', '15/04/2021', '15/04/2022', '15/08/2020',
       '15/08/2021', '15/08/2022', '15/12/2020', '15/12/2021',
       '15/12/2022', '15/02/2020', '15/02/2022', '15/02/2021',
       '15/01/2020', '15/01/2021', '15/01/2022', '15/01/2023',
       '15/07/2020', '15/07/2021', '15/07/2022', '15/06/2021',
       '15/06/2020', '15/05/2020', '15/05/2021', '15/05/2022',
       '15/03/2020', '15/03/2022', '15/03/2021', '15/11/2020',
       '15/11/2021', '15/11/2022', '15/10/2020', '15/10/2021',
       '15/10/2022', '15/09/2020', '15/09/2021', '15/09/2022',
       '15/09/2019', '15/10/2019', '15/11/2019', '15/05/2019',
       '15/03/2019', '15/06/2019', '15/07/2019', '15/01/2019',
       '15/02/2019', '15/12/2019', '15/08/2019', '15/04/2019'],
      dtype=object)

In [33]:
months_combined

,index,date,federal_state_id,federal_state_name,professional_area_ID,professional_area,number_of_unemployed,number_of_job _seekers,reported_open_positions,reported_open_positions_moving_annual_total
0,1,15/04/2020,1,Schleswig-Holstein,0,Gesamt,0,0,0,0
1,2,15/04/2020,1,Schleswig-Holstein,11,"Land-, Tier-, Forstwirtschaftsberufe",293,468,92,258
2,3,15/04/2020,1,Schleswig-Holstein,111,Landwirtschaft,107,175,41,116
3,4,15/04/2020,1,Schleswig-Holstein,1110,Berufe in der Landwirtschaft (o.S.),88,149,37,101
4,5,15/04/2020,1,Schleswig-Holstein,11102,Landwirtschaft (o.S.) - Fachkraft,47,79,34,82
...,...,...,...,...,...,...,...,...,...,...
20821,20822,15/04/2019,16,Thüringen,94623,Requisite - Spezialist,7,11,0,0
20822,20823,15/04/2019,16,Thüringen,947,Museumstechnik und -management,20,41,5,14
20823,20824,15/04/2019,16,Thüringen,9470,Museumsberufe (o.S.),16,36,5,14
20824,20825,15/04/2019,16,Thüringen,94704,Museum (o.S.) - Experte,16,36,5,14


In [29]:
# Filter the rows where the values in the desired column have exactly 5 digits
sub_sub = months_combined[(months_combined['professional_area_ID'].astype(str).str.len() == 5) & (months_combined['professional_area_ID'].dtype == 'float64')]

In [31]:
months_combined['federal_state_name'].unique()

array(['Schleswig-Holstein', 'Hamburg', 'Niedersachsen', 'Bremen',
       'Nordrhein-Westfalen', 'Hessen', 'Rheinland-Pfalz',
       'Baden-Württemberg', 'Bayern', 'Saarland', 'Berlin', 'Brandenburg',
       'Mecklenburg-Vorpommern', 'Sachsen', 'Sachsen-Anhalt', 'Thüringen'],
      dtype=object)

In [30]:
sub_sub.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 77924 entries, 2 to 21346
Data columns (total 10 columns):
 #   Column                                       Non-Null Count  Dtype  
---  ------                                       --------------  -----  
 0   index                                        77924 non-null  int64  
 1   date                                         77924 non-null  object 
 2   federal_state_id                             0 non-null      float64
 3   federal_state_name                           77924 non-null  object 
 4   professional_area_ID                         77924 non-null  float64
 5   professional_area                            77924 non-null  object 
 6   number_of_unemployed                         77924 non-null  object 
 7   number_of_job _seekers                       77924 non-null  object 
 8   reported_open_positions                      77924 non-null  object 
 9   reported_open_positions_moving_annual_total  77924 non-null  object 
dty

In [41]:
#mean of unemployment
sub_sub['reported_open_positions'].mean()

158.83397029579916

In [53]:
months_combined.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 1022397 entries, 0 to 20825
Data columns (total 10 columns):
 #   Column                                       Non-Null Count    Dtype  
---  ------                                       --------------    -----  
 0   index                                        1022397 non-null  int64  
 1   date                                         1022397 non-null  object 
 2   federal_state_id                             1022397 non-null  int64  
 3   federal_state_name                           1022397 non-null  object 
 4   professional_area_ID                         1021629 non-null  float64
 5   professional_area                            1022397 non-null  object 
 6   number_of_unemployed                         1022397 non-null  object 
 7   number_of_job _seekers                       1022397 non-null  object 
 8   reported_open_positions                      1022389 non-null  object 
 9   reported_open_positions_moving_annual_total  102

In [35]:
from sql_functions import get_engine
# Specify which table within your database you want to push your data to. Give your table an unambiguous name.
table_name2 = 'labour_market_jobs'
schema = 'muc_analytics_22_3'
engine = get_engine()
# If the specified table doesn't exist yet, it will be create
# Write records stored in a dataframe to SQL database
if engine!=None:
    try:
        months_combined.to_sql(name=table_name2, # Name of SQL table
                        con=engine, # Engine or connection
                        if_exists='replace', # Drop the table before inserting new values 
                        schema=schema, # Use schmea that was defined earlier
                        index=False, # Write DataFrame index as a column
                        chunksize=5000, # Specify the number of rows in each batch to be written at a time
                        method='multi') # Pass multiple values in a single INSERT clause
        print(f"The {table_name2} table was imported successfully.")
    # Error handling
    except (Exception, psycopg2.DatabaseError) as error:
        print(error)
        engine = None

The labour_market_jobs table was imported successfully.
